In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from fbprophet import Prophet
import pystan
import geopandas as gpd

C:\Users\user\anaconda3\envs\env_3.7\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
year_ds=['2015','2016','2017','2018','2019','2020']

In [3]:
df_1=pd.read_csv('./1인/1.csv',encoding='cp949')

df_1

df_1['Unnamed: 0'] = df_1['Unnamed: 0'].str.replace('?', '')
df_1.iloc[0,:] = df_1.iloc[0,:].str.replace(',', '')
df_1[['2015','2016','2017','2018','2019','2020']]=df_1[['2015','2016','2017','2018','2019','2020']].astype(int)
#df_1.info()

C:\Users\user\anaconda3\envs\env_3.7\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """


In [5]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from fbprophet import Prophet
import pystan
import geopandas as gpd
import os
os.chdir('D:/enc_3.7/env_3.7Dir/수질오염방제/입지분석')
import fnmatch

### 데이터정제

fd_list = fnmatch.filter(os.listdir('.'), '*.csv')
water_ = pd.DataFrame()
for i in range(len(fd_list)):
    tmp = pd.read_csv(fd_list[i], encoding = 'cp949', usecols=[0,1], header = None).iloc[1:,:]
    tmp[1] = tmp[1].str.replace(',', '')
    tmp[1] = tmp[1].astype(int)
    tmp[1]=tmp[1]*list(df_1.loc[1][1:])[i]
    year_ds[i] = pd.to_datetime(year_ds[i])
    tmp['ds'] = year_ds[i].replace(month=12,day=31)
    water_ = water_.append(tmp)

water_.columns = ['emd_nm','y','ds']
#water_['y'] = water_['y'].str.replace(',', '')
#water_['y'] = water_['y'].astype(int)

emd = water_['emd_nm'].unique()
water_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108 entries, 1 to 18
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   emd_nm  108 non-null    object        
 1   y       108 non-null    int32         
 2   ds      108 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(1)
memory usage: 3.0+ KB


### 이상치  필터링

In [47]:
sem_f = water_.copy(deep= True)
emd_cnt = sem_f.emd_nm.unique()

empty_df = pd.DataFrame(columns=['emd_nm', 'sem'])
for i in range(len(emd_cnt)):
    sem_val = sem_f[sem_f['emd_nm'] == emd_cnt[i]]
    empty_df = empty_df.append(pd.DataFrame([[emd_cnt[i], sem_val.sem()[0]]], columns=['emd_nm', 'sem']), ignore_index=True)
    
df = empty_df.sort_values('sem', ascending = False).reset_index(drop= True)

C:\Users\user\anaconda3\envs\env_3.7\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


In [48]:
df1 = df.copy(deep=True)
sem_m =  df1['sem'].mean()
df2 = df1[df1['sem'] > sem_m]
error = df2['emd_nm'].unique()

## prophet(시계열분석)

In [49]:
water_0 = water_[water_['emd_nm'] == emd[0]]
water_0 = water_0.loc[:,['ds','y']] 
prophet = Prophet(yearly_seasonality=True,
                  weekly_seasonality=False,
                  daily_seasonality=False,
                  changepoint_prior_scale=0.05)
prophet.fit(water_0)
future = prophet.make_future_dataframe(periods=3, freq='y')
forecast = prophet.predict(future)
fore = forecast.iloc[[-3,-2,-1],[0,15]]
fore.columns = ['ds','y']
result = fore.copy(deep=True)
result['emd_nm'] = emd[0]
for i in range(1, len(emd)):
    water = water_[water_['emd_nm'] == emd[i]]
    water = water.loc[:,['ds','y']] 
    prophet = Prophet(yearly_seasonality=True,
                      weekly_seasonality=False,
                      daily_seasonality=False,
                      changepoint_prior_scale=0.05)
    prophet.fit(water)
    future = prophet.make_future_dataframe(periods=3, freq='y')
    forecast = prophet.predict(future)
    fore = forecast.iloc[[-3,-2,-1],[0,15]]
    fore.columns = ['ds','y']
    result2 = fore.copy(deep=True)
    result2['emd_nm'] = emd[i]
    result = pd.concat([result, result2], axis = 0)

INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbproph

In [50]:
water_er_0 = water_[water_['emd_nm'] == error[0]]
water_er_0 = water_er_0.loc[:,['ds','y']] 
prophet = Prophet(yearly_seasonality=True,
                  weekly_seasonality=False,
                  daily_seasonality=False,
                  changepoint_prior_scale=0.07)
prophet.fit(water_er_0)
future = prophet.make_future_dataframe(periods=3, freq='y')
forecast = prophet.predict(future)
fore = forecast.iloc[[-3,-2,-1],[0,15]]
fore.columns = ['ds','y']
result3 = fore.copy(deep=True)
result3['emd_nm'] = error[0]
for i in range(1, len(error)):
    water = water_[water_['emd_nm'] == error[i]]
    water = water.loc[:,['ds','y']] 
    prophet = Prophet(yearly_seasonality=True,
                      weekly_seasonality=False,
                      daily_seasonality=False,
                      changepoint_prior_scale=0.07)
    prophet.fit(water)
    future = prophet.make_future_dataframe(periods=3, freq='y')
    forecast = prophet.predict(future)
    fore = forecast.iloc[[-3,-2,-1],[0,15]]
    fore.columns = ['ds','y']
    result4 = fore.copy(deep=True)
    result4['emd_nm'] = error[i]
    result3 = pd.concat([result3, result4], axis = 0)

INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.
INFO:fbprophet:n_changepoints greater than number of observations. Using 3.


In [51]:
result2 = pd.merge(result, result3,  on = ['emd_nm','ds'], how = 'outer')
result2.columns = ['ds','x','emd_nm','y']
result2['y']=result2['y'].fillna(result2['x'])
result2 = result2.drop('x',axis=1)
result2['y'] = round(result2['y']) 
result2['ds']= result2.ds.dt.year
result2 = result2.pivot(index='emd_nm', columns='ds', values='y')
result2.columns = ['y' + result2.columns[0].astype(str), 'y' + result2.columns[1].astype(str), 'y' + result2.columns[2].astype(str)]

### 수요예측결과 SHP

In [52]:
os.chdir('D:/enc_3.7/env_3.7Dir/수질오염방제/입지분석/fb_shp')
file_path ='SIG.shp'
emd = gpd.read_file(file_path,encoding='cp949')
emd=emd.rename({'SIG_KOR_NM':'emd_nm'},axis=1)
emd

INFO:fiona.ogrext:Failed to auto identify EPSG: 7


,SIG_CD,SIG_ENG_NM,emd_nm,geometry
0,11110,Jongno-gu,종로구,"POLYGON ((956615.453 1953567.199, 956621.579 1..."
1,11140,Jung-gu,중구,"POLYGON ((957890.386 1952616.746, 957909.908 1..."
2,11170,Yongsan-gu,용산구,"POLYGON ((953115.761 1950834.084, 953114.206 1..."
3,11200,Seongdong-gu,성동구,"POLYGON ((959681.109 1952649.605, 959842.412 1..."
4,11215,Gwangjin-gu,광진구,"POLYGON ((964825.058 1952633.250, 964875.565 1..."
...,...,...,...,...
245,48870,Hamyang-gun,함양군,"POLYGON ((1017715.723 1751280.191, 1017761.619..."
246,48880,Geochang-gun,거창군,"POLYGON ((1034571.593 1767759.812, 1034559.208..."
247,48890,Hapcheon-gun,합천군,"POLYGON ((1054066.576 1759517.415, 1054101.450..."
248,50110,Jeju-si,제주시,"MULTIPOLYGON (((874410.953 1480515.382, 874408..."


In [53]:

result_emd = pd.merge(emd, result2, on = ['emd_nm'], how = 'inner')
result_emd.to_file("시계열 물 사용 예측결과.shp", encoding = 'cp949')